In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install wfdb scipy scikit-learn numpy pandas matplotlib seaborn
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import os
import ast
from scipy.signal import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import wfdb
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import warnings
warnings.filterwarnings('ignore')

In [5]:
# --- 1. Data Loading ---
athlete_path = '/content/drive/MyDrive/NorwegianAthleteECG'
hcm_path = '/content/drive/MyDrive/ptb-xl'

def load_athlete_data(data_path):
    ecg_data = []
    for fname in sorted(os.listdir(data_path)):
        if fname.endswith('.dat'):
            rec_name = os.path.splitext(fname)[0]
            record = wfdb.rdsamp(os.path.join(data_path, rec_name))
            ecg_data.append(record[0])
    return ecg_data, ['athlete'] * len(ecg_data)

def load_hcm_data(data_path, csv_filename='ptbxl_database.csv'):
    meta = pd.read_csv(os.path.join(data_path, csv_filename))
    hcm_data = []
    for _, row in meta.iterrows():
        try:
            codes = ast.literal_eval(row['scp_codes'])
            if any('HYP' in str(code) for code in codes.keys()):
                rec_path = os.path.join(data_path, row['filename_hr'])
                record = wfdb.rdsamp(rec_path)
                hcm_data.append(record[0])
        except Exception:
            continue
    return hcm_data, ['hcm'] * len(hcm_data)

In [6]:
# --- 2. Data Preparation ---
athlete_data_list, athlete_labels = load_athlete_data(athlete_path)
hcm_data_list, hcm_labels = load_hcm_data(hcm_path)
hcm_data_resampled_list = [resample(ecg, 5000, axis=0) for ecg in hcm_data_list]
all_data = np.concatenate([np.array(athlete_data_list), np.array(hcm_data_resampled_list)])
all_labels = np.concatenate([athlete_labels, hcm_labels])
X = all_data
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(all_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print(f" Data Training shape: {X_train_scaled.shape}")


 Data Training shape: (39, 5000, 12)


In [7]:
# --- 3. The Hybrid LSTM-ONN Model Definition ---
def build_hybrid_model(hp):
    # Tunable Hyperparameters
    hp_lstm_units = hp.Int('lstm_units', min_value=32, max_value=96, step=32)
    hp_dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])

    # --- Part 1: Standard LSTM Feature Extractor ---
    input_layer = Input(shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]))
    # LSTM processes the sequence and outputs a single feature vector
    lstm_output = LSTM(units=hp_lstm_units)(input_layer)
    lstm_output = Dropout(hp_dropout)(lstm_output)
    lstm_output = BatchNormalization()(lstm_output)

    # --- Part 2: Simplified Oscillatory Layer (as a Dense layer) ---
    output_layer = Dense(64, activation='relu')(lstm_output)
    output_layer = Dense(1, activation='sigmoid')(output_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
# --- 4. Hyperparameter Search ---
tuner = kt.Hyperband(build_hybrid_model,
                     objective='val_accuracy',
                     max_epochs=25,
                     factor=3,
                     directory='my_dir',
                     project_name='ecg_hybrid_tuning')

stop_early = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Optimal parameters for Hybrid Model:
- LSTM Units: {best_hps.get('lstm_units')}
- Dropout Rate: {best_hps.get('dropout'):.2f}
- Learning Rate: {best_hps.get('learning_rate')}
""")

Trial 22 Complete [00h 00m 35s]
val_accuracy: 0.875

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 08m 54s

Optimal parameters for Hybrid Model:
- LSTM Units: 32
- Dropout Rate: 0.40
- Learning Rate: 0.01



In [9]:
# --- 5. Train and Evaluate the Final Hybrid Model ---
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])
loss, accuracy = final_model.evaluate(X_test_scaled, y_test)
print(f"\n Final Hybrid Model Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

predictions_prob = final_model.predict(X_test_scaled)
predictions = (predictions_prob > 0.5).astype(int).flatten()
print("\n Classification Report:")
print(classification_report(y_test, predictions, target_names=label_encoder.classes_))
print("\n Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.5806 - loss: 0.6573 - val_accuracy: 0.8750 - val_loss: 0.4693
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.9032 - loss: 0.3386 - val_accuracy: 0.8750 - val_loss: 0.3420
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9677 - loss: 0.1835 - val_accuracy: 0.8750 - val_loss: 0.2758
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9677 - loss: 0.0782 - val_accuracy: 0.8750 - val_loss: 0.2398
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0574 - val_accuracy: 0.8750 - val_loss: 0.2649
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0249 - val_accuracy: 0.8750 - val_loss: 0.2551
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0101 - val_accuracy: 0.8750 - val_loss: 0.2452
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0112 - val_accuracy: 0.8750 - val_loss: 0.2391
Epoch 9/50
1/1 ━

In [10]:
# --- 6. Original pure ONN model (low accuracy) ---

import numpy as np
import pandas as pd
import os
import ast
import random
from scipy.signal import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import wfdb
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

# --- 1. Data Loading (Same as before) ---
athlete_path = '/content/drive/MyDrive/NorwegianAthleteECG'
hcm_path = '/content/drive/MyDrive/ptb-xl'

def load_athlete_data(data_path):
    ecg_data = []
    for fname in sorted(os.listdir(data_path)):
        if fname.endswith('.dat'):
            rec_name = os.path.splitext(fname)[0]
            record = wfdb.rdsamp(os.path.join(data_path, rec_name))
            ecg_data.append(record[0])
    return ecg_data, ['athlete'] * len(ecg_data)

def load_hcm_data(data_path, csv_filename='ptbxl_database.csv'):
    meta = pd.read_csv(os.path.join(data_path, csv_filename))
    hcm_data = []
    for _, row in meta.iterrows():
        try:
            codes = ast.literal_eval(row['scp_codes'])
            if any('HYP' in str(code) for code in codes.keys()):
                rec_path = os.path.join(data_path, row['filename_hr'])
                record = wfdb.rdsamp(rec_path)
                hcm_data.append(record[0])
        except Exception:
            continue
    return hcm_data, ['hcm'] * len(hcm_data)

# --- 2. Data Processing and Preparation (same as before)
athlete_data_list, athlete_labels = load_athlete_data(athlete_path)
hcm_data_list, hcm_labels = load_hcm_data(hcm_path)
hcm_data_resampled_list = [resample(ecg, 5000, axis=0) for ecg in hcm_data_list]
all_data = np.concatenate([np.array(athlete_data_list), np.array(hcm_data_resampled_list)])
all_labels = np.concatenate([athlete_labels, hcm_labels])
X = all_data
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(all_labels)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42, stratify=y_train_full)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print(f"✅ Data prepared. Shapes: Train={X_train_scaled.shape}, Val={X_val_scaled.shape}, Test={X_test_scaled.shape}")


# --- 2. The Oscillatory Recurrent Neural Network (ORNN) ---
class OscillatoryRNN:
    def __init__(self, num_oscillators=8, lr=0.001, alpha=0.01, chunk_size=50):
        self.num_oscillators = num_oscillators
        self.lr, self.alpha = lr, alpha
        self.chunk_size = chunk_size

        # RNN weights
        self.W_xh = np.random.normal(0, 0.1, (12, num_oscillators)) # Input chunk (12 leads) to hidden (oscillators)
        self.W_hh = np.random.normal(0, 0.1, (num_oscillators, num_oscillators))

        # Oscillator dynamics parameters
        self.frequencies = np.random.uniform(0.5, 1.5, num_oscillators)
        self.coupling_matrix = np.random.uniform(-0.1, 0.1, (num_oscillators, num_oscillators))

        # Output layer
        self.W_out = np.random.normal(0, 0.1, (num_oscillators, 2)) # Final phases to class scores

    def _temporal_pool(self, ecg_signal):
        num_chunks = ecg_signal.shape[0] // self.chunk_size
        pooled_signal = np.array([np.mean(chunk, axis=0) for chunk in np.array_split(ecg_signal, num_chunks)])
        return pooled_signal

    def kuramoto_dynamics(self, phases, coupling_strength=0.1):
        phase_diffs = phases[:, None] - phases[None, :]
        coupling = coupling_strength * np.sum(self.coupling_matrix * np.sin(phase_diffs), axis=1)
        return self.frequencies + coupling

    def forward_pass(self, ecg_signal):
        pooled_input = self._temporal_pool(ecg_signal)
        self.phases_history = [] # Store for backpropagation
        phases = np.random.uniform(0, 2 * np.pi, self.num_oscillators)

        for input_chunk in pooled_input:
            input_influence = np.dot(input_chunk, self.W_xh)
            recurrent_influence = np.dot(np.sin(phases), self.W_hh) # Use sin(phases) for bounded recurrence
            kuramoto_influence = self.kuramoto_dynamics(phases)

            # Update phases (Euler integration step)
            phases += 0.01 * (input_influence + recurrent_influence + kuramoto_influence)
            phases %= (2 * np.pi)
            self.phases_history.append(phases.copy())

        final_phases = self.phases_history[-1]
        output_scores = np.dot(final_phases, self.W_out)

        # Softmax for probabilities
        e = np.exp(output_scores - np.max(output_scores))
        probabilities = e / np.sum(e)

        return probabilities, final_phases

    def train_step(self, ecg_signal, target_label):
        probabilities, final_phases = self.forward_pass(ecg_signal)
        target_one_hot = np.zeros(2); target_one_hot[target_label] = 1

        # Gradient of the loss w.r.t. the output scores
        output_grad = probabilities - target_one_hot

        # Backpropagate to output weights
        grad_W_out = np.outer(final_phases, output_grad)
        self.W_out -= self.lr * (grad_W_out + self.alpha * self.W_out)

        # Backpropagate error to the final phase state
        error_to_phases = np.dot(self.W_out, output_grad)

        # Simplified BPTT: Update recurrent weights based on final error
        prev_phases = self.phases_history[-2] if len(self.phases_history) > 1 else np.zeros_like(final_phases)
        grad_W_hh = np.outer(np.sin(prev_phases), error_to_phases)
        self.W_hh -= self.lr * (grad_W_hh + self.alpha * self.W_hh)

        # Update input weights based on the influence of all inputs
        avg_pooled_input = np.mean(self._temporal_pool(ecg_signal), axis=0)
        grad_W_xh = np.outer(avg_pooled_input, error_to_phases)
        self.W_xh -= self.lr * (grad_W_xh + self.alpha * self.W_xh)

    def predict(self, ecg_signal):
        probs, _ = self.forward_pass(ecg_signal)
        return np.argmax(probs)

# --- 3. Hyperparameter Search ---
param_space = {
    'num_oscillators': [4, 8, 12],
    'lr': [0.01, 0.005, 0.001],
    'alpha': [0.01, 0.001],
    'chunk_size': [25, 50, 100] # Controls sequence length
}
num_trials = 20
best_accuracy = -1
best_params = {}

for trial in range(num_trials):
    params = {key: random.choice(values) for key, values in param_space.items()}
    print(f"\nTrial {trial + 1}/{num_trials}: {params} ---")

    model = OscillatoryRNN(**params)
    for epoch in range(15):
        for i in np.random.permutation(len(X_train_scaled)):
            model.train_step(X_train_scaled[i], y_train[i])

    val_preds = [model.predict(x) for x in X_val_scaled]
    accuracy = accuracy_score(y_val, val_preds)
    print(f"Validation Accuracy: {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f"\n\n Best validation accuracy: {best_accuracy:.4f}")
print(f"Best parameters found: {best_params}")

# --- 4. Train Final Model and Evaluate ---
final_model = OscillatoryRNN(**best_params)
X_train_full_scaled = scaler.fit_transform(X_train_full.reshape(-1, X_train_full.shape[-1])).reshape(X_train_full.shape)
for epoch in range(40):
    for i in tqdm(np.random.permutation(len(X_train_full_scaled)), desc=f"Final Epoch {epoch+1}/40"):
        final_model.train_step(X_train_full_scaled[i], y_train_full[i])

test_predictions = [final_model.predict(x) for x in tqdm(X_test_scaled, desc="Final Evaluation")]
final_accuracy = accuracy_score(y_test, test_predictions)
print(f"\n Final ORNN Model Accuracy: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")
print("\n Classification Report:")
print(classification_report(y_test, test_predictions, target_names=label_encoder.classes_))
print("\n Confusion Matrix:")
print(confusion_matrix(y_test, test_predictions))

✅ Data prepared. Shapes: Train=(29, 5000, 12), Val=(10, 5000, 12), Test=(18, 5000, 12)

Trial 1/20: {'num_oscillators': 4, 'lr': 0.005, 'alpha': 0.001, 'chunk_size': 100} ---
Validation Accuracy: 0.5000

Trial 2/20: {'num_oscillators': 4, 'lr': 0.005, 'alpha': 0.01, 'chunk_size': 25} ---
Validation Accuracy: 0.5000

Trial 3/20: {'num_oscillators': 4, 'lr': 0.01, 'alpha': 0.01, 'chunk_size': 50} ---
Validation Accuracy: 0.4000

Trial 4/20: {'num_oscillators': 12, 'lr': 0.01, 'alpha': 0.01, 'chunk_size': 100} ---
Validation Accuracy: 0.6000

Trial 5/20: {'num_oscillators': 8, 'lr': 0.001, 'alpha': 0.01, 'chunk_size': 25} ---
Validation Accuracy: 0.5000

Trial 6/20: {'num_oscillators': 4, 'lr': 0.005, 'alpha': 0.01, 'chunk_size': 25} ---
Validation Accuracy: 0.5000

Trial 7/20: {'num_oscillators': 4, 'lr': 0.01, 'alpha': 0.001, 'chunk_size': 100} ---
Validation Accuracy: 0.5000

Trial 8/20: {'num_oscillators': 8, 'lr': 0.001, 'alpha': 0.001, 'chunk_size': 100} ---
Validation Accuracy: 0.6

Final Epoch 1/40:   0%|          | 0/39 [00:00<?, ?it/s]

Final Epoch 2/40:   0%|          | 0/39 [00:00<?, ?it/s]

Final Epoch 3/40:   0%|          | 0/39 [00:00<?, ?it/s]

Final Epoch 4/40:   0%|          | 0/39 [00:00<?, ?it/s]

...

Final Epoch 38/40:   0%|          | 0/39 [00:00<?, ?it/s]

Final Epoch 39/40:   0%|          | 0/39 [00:00<?, ?it/s]

Final Epoch 40/40:   0%|          | 0/39 [00:00<?, ?it/s]

Final Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]


 Final ORNN Model Accuracy: 0.5556 (55.56%)

 Classification Report:
              precision    recall  f1-score   support

     athlete       0.67      0.22      0.33         9
         hcm       0.53      0.89      0.67         9

    accuracy                           0.56        18
   macro avg       0.60      0.56      0.50        18
weighted avg       0.60      0.56      0.50        18


 Confusion Matrix:
[[2 7]
 [1 8]]
